In [1]:
import win32evtlog
import sys
from xml.etree import ElementTree as ET


server = 'localhost'
logtype = 'Security'

In [5]:
def parse_XML_log(event):
    tree = ET.ElementTree(ET.fromstring(event))
    root = tree.getroot()
    ns = "{http://schemas.microsoft.com/win/2004/08/events/event}"
    data = {}
    for eventID in root.findall(".//"):        
        if eventID.tag == f"{ns}System":
            for e_id in eventID.iter():
                if e_id.tag == f"{ns}System":
                    pass
                elif e_id.tag == f"{ns}Provider":
                    data["Provider"] = e_id.attrib.get('Name')                    
                elif e_id.tag == f"{ns}TimeCreated":
                    data["TimeCreated"] = e_id.attrib.get('SystemTime')                    
                elif e_id.tag == f"{ns}Correlation":
                    data["ActivityID"] = e_id.attrib.get('ActivityID')                   
                elif e_id.tag == f"{ns}Execution":
                    data["ProcessID"] = e_id.attrib.get('ProcessID')
                    data["ThreadID"] = e_id.attrib.get('ThreadID')                   
                else:
                    att = e_id.tag.replace(f"{ns}", "")
                    data[att] = e_id.text                  
        
        if eventID.tag == f"{ns}EventData":
            for attr in eventID.iter():
                if attr.tag == f'{ns}Data':                    
                    if attr.get('Name') is None:
                        data["Data"] = attr.text
                    else:
                        data[attr.get('Name')] = attr.text
                elif attr.tag == f'{ns}Binary':
                    data["Binary"] = attr.text 

    return data

In [3]:
event_context = { "info": "this object is always passed to your callback" }
# Event log source to listen to
event_source = 'application'

def new_logs_event_handler(reason, context, evt):
    """
    Called when new events are logged.

    reason - reason the event was logged?
    context - context the event handler was registered with
    evt - event handle
    """
    # Just print some information about the event
    # print ('reason', reason, 'context', context, 'event handle', evt)

    event = win32evtlog.EvtRender(evt, win32evtlog.EvtRenderEventXml)
    result = " ".join(l.strip() for l in event.splitlines())
    print(result)
    with open('xml_logs.txt', 'a') as file:
        file.write(result)
        file.write('\n')
    # parse_XML_log(event=event)
    print(' - ')

    # Make sure all printed text is actually printed to the console now
    sys.stdout.flush()
    return 0

In [ ]:


# Subscribe to future events
subscription1 = win32evtlog.EvtSubscribe(event_source, win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)
subscription2 = win32evtlog.EvtSubscribe('system', win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)
subscription3 = win32evtlog.EvtSubscribe('Security', win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)


<Event xmlns='http://schemas.microsoft.com/win/2004/08/events/event'><System><Provider Name='MYEVENTSOURCE'/><EventID Qualifiers='0'>1</EventID><Version>0</Version><Level>4</Level><Task>0</Task><Opcode>0</Opcode><Keywords>0x80000000000000</Keywords><TimeCreated SystemTime='2023-05-29T13:07:21.4322380Z'/><EventRecordID>124336</EventRecordID><Correlation/><Execution ProcessID='0' ThreadID='0'/><Channel>Application</Channel><Computer>DESKTOP-1LIHC4I</Computer><Security UserID='S-1-5-21-2384755564-1941788603-3002086374-1001'/></System><EventData><Data>My first log</Data></EventData></Event>
 - 
<Event xmlns='http://schemas.microsoft.com/win/2004/08/events/event'><System><Provider Name='Microsoft-Windows-Security-Auditing' Guid='{54849625-5478-4994-a5ba-3e3b0328c30d}'/><EventID>4798</EventID><Version>0</Version><Level>0</Level><Task>13824</Task><Opcode>0</Opcode><Keywords>0x8020000000000000</Keywords><TimeCreated SystemTime='2023-05-29T13:07:25.6698357Z'/><EventRecordID>479686</EventRecordI

In [6]:
data = []
with open("xml_logs.txt", "r+") as file1:
    lines = file1.readlines()   
    for line in lines:              
        data.append(parse_XML_log(line))   
        
data


[{'Provider': 'Microsoft-Windows-Security-Auditing',
  'EventID': '4798',
  'Version': '0',
  'Level': '0',
  'Task': '13824',
  'Opcode': '0',
  'Keywords': '0x8020000000000000',
  'TimeCreated': '2023-05-27T11:07:27.0563057Z',
  'EventRecordID': '470695',
  'ActivityID': '{b4860852-8edd-0002-c708-86b4dd8ed901}',
  'ProcessID': '680',
  'ThreadID': '844',
  'Channel': 'Security',
  'Computer': 'DESKTOP-1LIHC4I',
  'Security': None,
  'TargetUserName': 'Louki',
  'TargetDomainName': 'DESKTOP-1LIHC4I',
  'TargetSid': 'S-1-5-21-2384755564-1941788603-3002086374-1001',
  'SubjectUserSid': 'S-1-5-21-2384755564-1941788603-3002086374-1001',
  'SubjectUserName': 'Louki',
  'SubjectDomainName': 'DESKTOP-1LIHC4I',
  'SubjectLogonId': '0x63a1c67',
  'CallerProcessId': '0x2b74',
  'CallerProcessName': 'C:\\Program Files\\Git\\usr\\bin\\sh.exe'},
 {'Provider': 'Microsoft-Windows-Security-Auditing',
  'EventID': '5379',
  'Version': '0',
  'Level': '0',
  'Task': '13824',
  'Opcode': '0',
  'Keyword

In [7]:
for list in data:
    print(list)

{'Provider': 'Microsoft-Windows-Security-Auditing', 'EventID': '4798', 'Version': '0', 'Level': '0', 'Task': '13824', 'Opcode': '0', 'Keywords': '0x8020000000000000', 'TimeCreated': '2023-05-27T11:07:27.0563057Z', 'EventRecordID': '470695', 'ActivityID': '{b4860852-8edd-0002-c708-86b4dd8ed901}', 'ProcessID': '680', 'ThreadID': '844', 'Channel': 'Security', 'Computer': 'DESKTOP-1LIHC4I', 'Security': None, 'TargetUserName': 'Louki', 'TargetDomainName': 'DESKTOP-1LIHC4I', 'TargetSid': 'S-1-5-21-2384755564-1941788603-3002086374-1001', 'SubjectUserSid': 'S-1-5-21-2384755564-1941788603-3002086374-1001', 'SubjectUserName': 'Louki', 'SubjectDomainName': 'DESKTOP-1LIHC4I', 'SubjectLogonId': '0x63a1c67', 'CallerProcessId': '0x2b74', 'CallerProcessName': 'C:\\Program Files\\Git\\usr\\bin\\sh.exe'}
{'Provider': 'Microsoft-Windows-Security-Auditing', 'EventID': '5379', 'Version': '0', 'Level': '0', 'Task': '13824', 'Opcode': '0', 'Keywords': '0x8020000000000000', 'TimeCreated': '2023-05-27T11:07:27

In [ ]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['Windows_Event_Logs']
collection = db['mycollection']


# Insert the list into the collection

collection.insert_many(data)

# Print the inserted documents
for doc in collection.find():
    print(doc)

In [ ]:
import pyuac

def main():
    print("Do stuff here that requires being run as an admin.")
    # The window will disappear as soon as the program exits!
    input("Press enter to close the window. >")

if __name__ == "__main__":
    if not pyuac.isUserAdmin():
        print("Re-launching as admin!")
       
        pyuac.runAsAdmin()
        input("Press enter to close the window. >")
    else:        
        main()  # Already an admin here.

In [ ]:
import json
import xmltodict

def xml_to_json(xml_string):
    xml_dict = xmltodict.parse(xml_string)
    json_data = json.dumps(xml_dict)
    return json_data

In [ ]:
js = xml_to_json(lines[0])
js

In [ ]:
with open('test.json', 'w') as f:
    f.write(js)